In [1]:
import rasterio
import rasterio.features
import rasterio.warp
#import geopandas as gpd
#import seaborn as sns
#import matplotlib.pyplot
import numpy as np
import pandas as pd
import os

In [2]:
df = pd.read_csv('data_processed.csv')

In [3]:
#x: longitud y: latitud
x = -72.6983	
y = -42.4087
df

,Región,Comuna,Tipo Remoción en masa,Factor desencadenante,Fecha Evento,Longitud,Latitud,Sistema Georeferencia,Cota (m.s.n.m)
0,Los Lagos,Castro,Flujo,Precipitaciones,07/03/2001,-73.7855,-42.4686,WGS 84- UTM 18 S,-
1,Los Lagos,Castro,Flujo,Precipitaciones,09/03/2001,-73.7855,-42.4686,WGS 84- UTM 18 S,-
2,Los Lagos,Chaitén,Deslizamiento,Precipitaciones,03/05/2002,-72.6983,-42.4087,WGS 84- UTM 18 S,-
3,Los Lagos,Río Negro,Deslizamiento,Precipitaciones,09/08/2002,-73.7889,-40.7083,WGS 84- UTM 18 S,6
4,Los Lagos,Osorno,Deslizamiento,Precipitaciones,22/08/2007,-73.6047,-40.5750,WGS 84- UTM 18 S,126
...,...,...,...,...,...,...,...,...,...
1242,Biobío,Tomé,Deslizamiento,Precipitaciones,11/07/2006,-72.9781,-36.5932,WGS 84/ UTM 18 S,40
1243,Biobío,Talcahuano,Caída,Precipitaciones,15/06/1991,-73.1032,-36.7543,WGS 84/ UTM 18 S,-
1244,Biobío,Talcahuano,Caída,Precipitaciones,16/06/1991,-73.1032,-36.7543,WGS 84/ UTM 18 S,-
1245,Biobío,Talcahuano,Caída,Precipitaciones,17/06/1991,-73.1032,-36.7543,WGS 84/ UTM 18 S,-


In [4]:
#Contar tipos de valores obtenido en "Tipo Remoción en masa"
conteo_tipos_remocion = df["Tipo Remoción en masa"].value_counts()

print("Conteo de valores en la columna 'Tipo de remoción en masa':")
print(conteo_tipos_remocion)


Conteo de valores en la columna 'Tipo de remoción en masa':
Tipo Remoción en masa
Deslizamiento           978
Flujo                   153
Caída                    79
Deslizamiento, Flujo     13
Deslizamiento. Flujo     11
Deslizameinto             4
Caída de bloques          4
deslizamiento             2
Inundación                2
Deslizamient, Flujo       1
Name: count, dtype: int64


In [5]:
path = r'ROSETTA_MEAN/theta_r_15-30cm.tif'

path_PIRange = r'PIRange-001'
path_rosetta_mean = r'ROSETTA_MEAN'
path_rosetta_sd = r'ROSETTA_SD'
path_SoilMaps_mean = r'SoilMaps_MEAN'
#path_SoilMaps_mean_2 = r'SoilMaps_MEAN'
path_Textural_Classes = r'Textural_Classes'
all_paths = [path_PIRange, path_rosetta_mean, path_rosetta_sd, path_SoilMaps_mean, path_Textural_Classes] #path_SoilMaps_mean_2
all_paths

['PIRange-001',
 'ROSETTA_MEAN',
 'ROSETTA_SD',
 'SoilMaps_MEAN',
 'Textural_Classes']

In [6]:
def archivos_extraccion(paths):
    archivos = []
    for path in paths:
        for elemento in os.listdir(path):
            ruta_archivo = os.path.join(path, elemento)
            if os.path.isfile(ruta_archivo) and  ruta_archivo.lower().endswith('.tif'):
                archivos.append(fr'{path}/{elemento}')
    return archivos

archivos = archivos_extraccion(all_paths)
archivos[0:3]

['PIRange-001/PIRange_Bulkd.0-5cm.tif',
 'PIRange-001/PIRange_Bulkd.100-200cm.tif',
 'PIRange-001/PIRange_Bulkd.15-30cm.tif']

In [7]:
for archivo in archivos:
    print(archivo.split('/')[1])

PIRange_Bulkd.0-5cm.tif
PIRange_Bulkd.100-200cm.tif
PIRange_Bulkd.15-30cm.tif
PIRange_Bulkd.30-60cm.tif
PIRange_Bulkd.5-15cm.tif
PIRange_Bulkd.60-100cm.tif
PIRange_Clay.0-5cm.tif
PIRange_Clay.100-200cm.tif
PIRange_Clay.15-30cm.tif
PIRange_Clay.30-60cm.tif
PIRange_Clay.5-15cm.tif
PIRange_Clay.60-100cm.tif
PIRange_Sand.0-5cm.tif
PIRange_Sand.100-200cm.tif
PIRange_Sand.15-30cm.tif
PIRange_Sand.30-60cm.tif
PIRange_Sand.5-15cm.tif
PIRange_Sand.60-100cm.tif
alpha_0-5cm.tif
alpha_100-200cm.tif
alpha_15-30cm.tif
alpha_30-60cm.tif
alpha_5-15cm.tif
alpha_60-100cm.tif
AvMoist.0-5cm.tif
AvMoist.100-200cm.tif
AvMoist.15-30cm.tif
AvMoist.30-60cm.tif
AvMoist.5-15cm.tif
AvMoist.60-100cm.tif
AWC_0-5cm.tif
AWC_100-200cm.tif
AWC_15-30cm.tif
AWC_30-60cm.tif
AWC_5-15cm.tif
AWC_60-100cm.tif
FC.0-5cm.tif
FC.100-200cm.tif
FC.15-30cm.tif
FC.30-60cm.tif
FC.5-15cm.tif
FC.60-100cm.tif
ksat_0-5cm.tif
ksat_100-200cm.tif
ksat_15-30cm.tif
ksat_30-60cm.tif
ksat_5-15cm.tif
ksat_60-100cm.tif
n_0-5cm.tif
n_100-200cm.tif


In [8]:
"""with rasterio.open(path) as dataset:
    mask = dataset.dataset_mask()
    for geom, val in rasterio.features.shapes(
        mask, transform=dataset.transform):
        geom = rasterio.warp.transform_geom(dataset.crs, 'EPSG:4326', geom, precision=6)
        
    print(geom)"""

"with rasterio.open(path) as dataset:\n    mask = dataset.dataset_mask()\n    for geom, val in rasterio.features.shapes(\n        mask, transform=dataset.transform):\n        geom = rasterio.warp.transform_geom(dataset.crs, 'EPSG:4326', geom, precision=6)\n        \n    print(geom)"

In [15]:
#funcions de lectura de tif
#x_,y_ = dataset_.index(-71.838400,-39.554800)
path = r'ROSETTA_MEAN/theta_r_15-30cm.tif'
dataset_ = rasterio.open(path)
x_,y_ = dataset_.index(x,y)
#print(x,y)
#print(x_,y_)
#x: longitud y: latitud
banda=dataset_.read(1)

print(banda[x_][y_])
valor = banda[x_][y_]
#print('Valor:', type(valor))

0.087


In [16]:
nuevo = range(0,1247)
df_val = pd.DataFrame({'Nombre': nuevo})
pd.concat([df, df_val], axis=1)

,Región,Comuna,Tipo Remoción en masa,Factor desencadenante,Fecha Evento,Longitud,Latitud,Sistema Georeferencia,Cota (m.s.n.m),Nombre
0,Los Lagos,Castro,Flujo,Precipitaciones,07/03/2001,-73.7855,-42.4686,WGS 84- UTM 18 S,-,0
1,Los Lagos,Castro,Flujo,Precipitaciones,09/03/2001,-73.7855,-42.4686,WGS 84- UTM 18 S,-,1
2,Los Lagos,Chaitén,Deslizamiento,Precipitaciones,03/05/2002,-72.6983,-42.4087,WGS 84- UTM 18 S,-,2
3,Los Lagos,Río Negro,Deslizamiento,Precipitaciones,09/08/2002,-73.7889,-40.7083,WGS 84- UTM 18 S,6,3
4,Los Lagos,Osorno,Deslizamiento,Precipitaciones,22/08/2007,-73.6047,-40.5750,WGS 84- UTM 18 S,126,4
...,...,...,...,...,...,...,...,...,...,...
1242,Biobío,Tomé,Deslizamiento,Precipitaciones,11/07/2006,-72.9781,-36.5932,WGS 84/ UTM 18 S,40,1242
1243,Biobío,Talcahuano,Caída,Precipitaciones,15/06/1991,-73.1032,-36.7543,WGS 84/ UTM 18 S,-,1243
1244,Biobío,Talcahuano,Caída,Precipitaciones,16/06/1991,-73.1032,-36.7543,WGS 84/ UTM 18 S,-,1244
1245,Biobío,Talcahuano,Caída,Precipitaciones,17/06/1991,-73.1032,-36.7543,WGS 84/ UTM 18 S,-,1245


In [80]:
#print(new_df.Longitud.iloc[0])
#print(new_df.iloc[0].Longitud)

-73.7855
-73.7855


In [9]:
#x: longitud y: latitud
new_df = pd.concat([df.Longitud, df.Latitud], axis = 1)
#archivos = [path]

In [18]:
new_df

,Longitud,Latitud
0,-73.7855,-42.4686
1,-73.7855,-42.4686
2,-72.6983,-42.4087
3,-73.7889,-40.7083
4,-73.6047,-40.5750
...,...,...
1242,-72.9781,-36.5932
1243,-73.1032,-36.7543
1244,-73.1032,-36.7543
1245,-73.1032,-36.7543


In [45]:
arrs = [
        [(0.001,.0), (0.001,0.001), (.0,0.001), (-0.001,.0), (-0.001,-0.001), (.0,-0.001)]
    ]
x = 0
y = 0
for arr in arrs:
    for (py,px) in arr: 
        new_x = x + px
        new_y = y + py
        print(new_x, new_y)

0.0 0.001
0.001 0.001
0.001 0.0
0.0 -0.001
-0.001 -0.001
-0.001 0.0


In [11]:
len(archivos)

133

In [12]:
pd.read_csv('tif_data')

,Unnamed: 0,Longitud,Latitud,PIRange_Bulkd.0-5cm.tif,PIRange_Bulkd.100-200cm.tif,PIRange_Bulkd.15-30cm.tif,PIRange_Bulkd.30-60cm.tif,PIRange_Bulkd.5-15cm.tif,PIRange_Bulkd.60-100cm.tif,PIRange_Clay.0-5cm.tif,...,Silt.15-30cm.tif,Silt.30-60cm.tif,Silt.5-15cm.tif,Silt.60-100cm.tif,Tex_Class.0-5cm.tif,Tex_Class.100-200cm.tif,Tex_Class.15-30cm.tif,Tex_Class.30-60cm.tif,Tex_Class.5-15cm.tif,Tex_Class.60-100cm.tif
0,0,-73.7855,-42.4686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,-73.7855,-42.4686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,-72.6983,-42.4087,0.600,0.697,0.495,0.628,0.510,0.666,25.056002,...,18.825001,17.982000,16.952000,16.462999,8.0,8.0,8.0,8.0,8.0,8.0
3,3,-73.7889,-40.7083,0.636,0.705,0.531,0.662,0.550,0.667,29.791002,...,32.243999,31.320000,25.072001,29.937000,3.0,3.0,2.0,2.0,2.0,2.0
4,4,-73.6047,-40.5750,0.591,0.785,0.484,0.618,0.498,0.721,26.359001,...,20.194000,18.142000,21.920000,15.645000,3.0,8.0,7.0,7.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242,1242,-72.9781,-36.5932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1243,1243,-73.1032,-36.7543,0.593,0.695,0.486,0.620,0.514,0.664,29.827000,...,31.188999,28.318001,23.614000,27.635000,2.0,2.0,2.0,2.0,2.0,2.0
1244,1244,-73.1032,-36.7543,0.593,0.695,0.486,0.620,0.514,0.664,29.827000,...,31.188999,28.318001,23.614000,27.635000,2.0,2.0,2.0,2.0,2.0,2.0
1245,1245,-73.1032,-36.7543,0.593,0.695,0.486,0.620,0.514,0.664,29.827000,...,31.188999,28.318001,23.614000,27.635000,2.0,2.0,2.0,2.0,2.0,2.0


In [94]:
archivos

'PIRange-002/PIRange_Bulkd.0-5cm.tif'

In [147]:
archivos = [r'PIRange-002/PIRange_Bulkd.0-5cm.tif']

In [22]:
new_df

,Longitud,Latitud
0,-73.7855,-42.4686
1,-73.7855,-42.4686
2,-72.6983,-42.4087
3,-73.7889,-40.7083
4,-73.6047,-40.5750
...,...,...
1242,-72.9781,-36.5932
1243,-73.1032,-36.7543
1244,-73.1032,-36.7543
1245,-73.1032,-36.7543


In [27]:
new_df

,Longitud,Latitud
0,-73.7755,-42.4586
1,-73.7755,-42.4586
2,-72.6883,-42.3987
3,-73.7789,-40.6983
4,-73.5947,-40.5650
...,...,...
1242,-72.9681,-36.5832
1243,-73.0932,-36.7443
1244,-73.0932,-36.7443
1245,-73.0932,-36.7443


In [36]:
archivos = r'PIRange-001/PIRange_Bulkd.0-5cm.tif'
dataset_ = rasterio.open(archivos)
# -73.6855	-42.3686	
x = -73.7855
y = -42.4686
x_, y_ = dataset_.index(x,y)
print(x_,y_)
banda = dataset_.read()
print(nan_latlon(archivos,x,y))


25150 2209
valor de arr: [(0.001, 0.0), (0.001, 0.001), (0.0, 0.001), (-0.001, 0.0), (-0.001, -0.001), (0.0, -0.001)]
-73.7855 -42.467600000000004
nan


In [143]:
x = -73.6866
y = -42.3686
x_, y_ = dataset_.index(x,y)
banda = dataset_.read(1)
banda[x_,y_]

0.589

In [13]:
print(new_df.loc[new_df.Longitud > -70])
print(new_df.loc[df.Latitud > -30]) #eliminar 97, 128, 446
drop_rows = [97,128,446]
new_df = new_df.drop(new_df.index[drop_rows])


     Longitud    Latitud
446 -1.366385 -43.307927
      Longitud   Latitud
97  -73.760202 -7.080473
128 -73.630183 -0.008028


In [14]:
new_df += 0.01

In [15]:
from tqdm import tqdm

In [30]:
new_df.to_csv('raw_data_0_01')

In [25]:
def nan_latlon(dataset, x: float , y: float):
    # x += 1 , y += 0
    # x += 0 , y += 1
    # x += 1 , y += 1
    
    # x -= 1 , y -= 0
    # x -= 0 , y -= 1
    # x -= 1 , y -= 1
    arrs = [
        [(0.001,.0), (0.001,0.001), (.0,0.001), (-0.001,.0), (-0.001,-0.001), (.0,-0.001)]
    ]
    for arr in arrs:
        for (py, px) in arr:
            x_ = x + px
            y_ = y + py
            #print(x_, y_)
            x__, y__ = dataset.index(x_,y_)
            banda = dataset.read(1)
            try:
                valor = banda[x__,y__]
                if not np.isnan(valor):
                    return valor
            except:
                continue
    return np.nan

In [26]:
def tif_extraccion(archivos: list, latlon: pd.DataFrame):
    # archivos: [path1.tif, path2.tif, ...]
    # latlon: [Longitud , Latitud]
    #           x1          y1
    #           x2          y2
    #           ...         ...
    df = pd.DataFrame(latlon)
    avance = 0
    total = len(archivos)
    for archivo in tqdm(archivos):
        carga = avance / total
        #print(f'Avance de {carga*100}%')
        dataset = rasterio.open(archivo)
        nombre = archivo.split('/')[1]
        valores = []
        for i in range(len(latlon)):
            x = latlon.Longitud.iloc[i]
            y = latlon.Latitud.iloc[i]
            #print(f'Coordenadas: {x}, {y}')
            x_, y_ = dataset.index(x,y)
            banda = dataset.read(1)
            #valor= np.nan
            try:
                valor = banda[x_,y_]
                #print(f'valor: {valor}')
                if np.isnan(valor):
                    #print('dentro de if')
                    valor = nan_latlon(dataset=dataset, x=x, y=y)
                    #print(f'Valor post nan_latlon(): {valor}')
            except Exception as e:
                valor = nan_latlon(dataset=dataset, x=x, y=y)
                #print(f'valor except: {valor}')
            valores.append(valor)
        df[nombre] = valores
        avance += 1
    # return:
    # df: [Longitud, Latitud, tif1_data, tif2_data, ..., tifn_data]
    #       Long1      Lat1    data1_1    data1_2   ...   data1_n
    #       Long2      Lat2    data2_1    data2_2   ...   data2_n
    #       ...        ...     ...        ...       ...      ...
    return df

new_col = tif_extraccion(archivos, new_df)
#porcentaje = new_col['theta_r_15-30cm.tif'].isna().sum()
#print(f'Cantidad de datos: {len(new_col)}\nTiene un {porcentaje/len(new_col)}% de NaN')
new_col.to_csv('tif_data_nan_function_0_01')

  0%|          | 0/133 [03:01<?, ?it/s]


KeyboardInterrupt: 

In [165]:
pd.read_csv('tif_data_nan_function').isna().sum()

Unnamed: 0                   0
Longitud                     0
Latitud                      0
PIRange_Bulkd.0-5cm.tif    571
dtype: int64

In [138]:
import pandas as pd
pd.read_csv('tif_data_0_1').isna().sum()
pd.read_csv('tif_data_0_1')

,Unnamed: 0,Longitud,Latitud,PIRange_Bulkd.0-5cm.tif,PIRange_Bulkd.100-200cm.tif,PIRange_Bulkd.15-30cm.tif,PIRange_Bulkd.30-60cm.tif,PIRange_Bulkd.5-15cm.tif,PIRange_Bulkd.60-100cm.tif,PIRange_Clay.0-5cm.tif,...,Silt.15-30cm.tif,Silt.30-60cm.tif,Silt.5-15cm.tif,Silt.60-100cm.tif,Tex_Class.0-5cm.tif,Tex_Class.100-200cm.tif,Tex_Class.15-30cm.tif,Tex_Class.30-60cm.tif,Tex_Class.5-15cm.tif,Tex_Class.60-100cm.tif
0,0,-73.6855,-42.3686,0.591,0.683,0.485,0.615,0.521,0.649,28.473999,...,15.393000,14.903000,11.037000,10.456,7.0,7.0,8.0,8.0,8.0,8.0
1,1,-73.6855,-42.3686,0.591,0.683,0.485,0.615,0.521,0.649,28.473999,...,15.393000,14.903000,11.037000,10.456,7.0,7.0,8.0,8.0,8.0,8.0
2,2,-72.5983,-42.3087,0.604,0.693,0.493,0.628,0.531,0.665,18.759001,...,16.912001,16.551001,11.022000,12.853,8.0,4.0,8.0,4.0,4.0,4.0
3,3,-73.6889,-40.6083,0.631,0.692,0.482,0.677,0.582,0.663,24.823999,...,19.743000,18.921000,9.619000,19.087,8.0,8.0,8.0,8.0,7.0,8.0
4,4,-73.5047,-40.4750,0.597,0.729,0.490,0.620,0.550,0.688,25.062002,...,21.020000,19.746000,17.562000,16.875,7.0,7.0,7.0,7.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242,1242,-72.8781,-36.4932,0.605,0.704,0.509,0.626,0.514,0.667,24.642000,...,26.215000,19.839001,21.865999,17.573,7.0,7.0,7.0,7.0,7.0,6.0
1243,1243,-73.0032,-36.6543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1244,1244,-73.0032,-36.6543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1245,1245,-73.0032,-36.6543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_

,Unnamed: 0,Longitud,Latitud,PIRange_Bulkd.0-5cm.tif,PIRange_Bulkd.100-200cm.tif,PIRange_Bulkd.15-30cm.tif,PIRange_Bulkd.30-60cm.tif,PIRange_Bulkd.5-15cm.tif,PIRange_Bulkd.60-100cm.tif,PIRange_Clay.0-5cm.tif,...,Silt.15-30cm.tif,Silt.30-60cm.tif,Silt.5-15cm.tif,Silt.60-100cm.tif,Tex_Class.0-5cm.tif,Tex_Class.100-200cm.tif,Tex_Class.15-30cm.tif,Tex_Class.30-60cm.tif,Tex_Class.5-15cm.tif,Tex_Class.60-100cm.tif
0,0,-73.7845,-42.4676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,-73.7845,-42.4676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,-72.6973,-42.4077,0.603,0.701,0.498,0.632,0.513,0.679,24.934999,...,13.882000,13.503000,15.780000,11.821000,8.0,8.0,8.0,8.0,8.0,8.0
3,3,-73.7879,-40.7073,0.646,0.713,0.555,0.663,0.539,0.672,28.616001,...,29.629999,29.009001,21.861000,27.797001,3.0,3.0,2.0,2.0,2.0,2.0
4,4,-73.6037,-40.5740,0.592,0.772,0.485,0.618,0.498,0.716,25.787001,...,20.754000,18.902000,20.378000,16.431000,3.0,8.0,7.0,7.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242,1242,-72.9771,-36.5922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1243,1243,-73.1022,-36.7533,0.590,0.701,0.483,0.617,0.491,0.671,29.196999,...,31.297001,28.572001,21.292999,27.978001,2.0,1.0,2.0,2.0,1.0,2.0
1244,1244,-73.1022,-36.7533,0.590,0.701,0.483,0.617,0.491,0.671,29.196999,...,31.297001,28.572001,21.292999,27.978001,2.0,1.0,2.0,2.0,1.0,2.0
1245,1245,-73.1022,-36.7533,0.590,0.701,0.483,0.617,0.491,0.671,29.196999,...,31.297001,28.572001,21.292999,27.978001,2.0,1.0,2.0,2.0,1.0,2.0


In [154]:
new_df

,Longitud,Latitud
0,-73.7755,-42.4586
1,-73.7755,-42.4586
2,-72.6883,-42.3987
3,-73.7789,-40.6983
4,-73.5947,-40.5650
...,...,...
1242,-72.9681,-36.5832
1243,-73.0932,-36.7443
1244,-73.0932,-36.7443
1245,-73.0932,-36.7443


In [128]:
len(new_col.columns)

135

In [112]:
def calcular_porcentajes_nan(df, col):
    porcentaje = df[col].isna().sum()
    print(f'Cantidad de datos: {len(new_col)}\nTiene un {porcentaje/len(new_col)}% de NaN')

In [131]:
len(new_col.columns), len(archivos)

(135, 133)

In [129]:
calcular_porcentajes_nan(new_col, new_col.columns)

Cantidad de datos: 1247
Tiene un Longitud                       0.000000
Latitud                        0.000000
PIRange_Bulkd.0-5cm.tif        0.457899
PIRange_Bulkd.100-200cm.tif    0.457097
PIRange_Bulkd.15-30cm.tif      0.457899
                                 ...   
Tex_Class.100-200cm.tif        0.457899
Tex_Class.15-30cm.tif          0.457097
Tex_Class.30-60cm.tif          0.457097
Tex_Class.5-15cm.tif           0.457899
Tex_Class.60-100cm.tif         0.457097
Length: 135, dtype: float64% de NaN


In [115]:
len(new_col) * 0.45

561.15

In [98]:
new_col.to_csv('test_dataframe')

In [94]:
porcentaje = new_col['theta_r_15-30cm.tif'].isna().sum()
print(f'Cantidad de datos: {len(new_col)}\nTiene un {porcentaje/len(new_col)}% de NaN')

Cantidad de datos: 1247
Tiene un 0.4578989574979952% de NaN


In [145]:
df = new_df
df

,Longitud,Latitud
0,-73.7855,-42.4686
1,-73.7855,-42.4686
2,-72.6983,-42.4087
3,-73.7889,-40.7083
4,-73.6047,-40.5750
...,...,...
1242,-72.9781,-36.5932
1243,-73.1032,-36.7543
1244,-73.1032,-36.7543
1245,-73.1032,-36.7543


In [13]:
df.Longitud += 0.01
df.Latitud += 0.01
df

,Región,Comuna,Tipo Remoción en masa,Factor desencadenante,Fecha Evento,Longitud,Latitud,Sistema Georeferencia,Cota (m.s.n.m)
0,Los Lagos,Castro,Flujo,Precipitaciones,07/03/2001,-73.7755,-42.4586,WGS 84- UTM 18 S,-
1,Los Lagos,Castro,Flujo,Precipitaciones,09/03/2001,-73.7755,-42.4586,WGS 84- UTM 18 S,-
2,Los Lagos,Chaitén,Deslizamiento,Precipitaciones,03/05/2002,-72.6883,-42.3987,WGS 84- UTM 18 S,-
3,Los Lagos,Río Negro,Deslizamiento,Precipitaciones,09/08/2002,-73.7789,-40.6983,WGS 84- UTM 18 S,6
4,Los Lagos,Osorno,Deslizamiento,Precipitaciones,22/08/2007,-73.5947,-40.5650,WGS 84- UTM 18 S,126
...,...,...,...,...,...,...,...,...,...
1242,Biobío,Tomé,Deslizamiento,Precipitaciones,11/07/2006,-72.9681,-36.5832,WGS 84/ UTM 18 S,40
1243,Biobío,Talcahuano,Caída,Precipitaciones,15/06/1991,-73.0932,-36.7443,WGS 84/ UTM 18 S,-
1244,Biobío,Talcahuano,Caída,Precipitaciones,16/06/1991,-73.0932,-36.7443,WGS 84/ UTM 18 S,-
1245,Biobío,Talcahuano,Caída,Precipitaciones,17/06/1991,-73.0932,-36.7443,WGS 84/ UTM 18 S,-


In [10]:

new_df = pd.concat([df.Longitud, df.Latitud], axis = 1)


In [144]:
new_df

,Longitud,Latitud
0,-73.7855,-42.4686
1,-73.7855,-42.4686
2,-72.6983,-42.4087
3,-73.7889,-40.7083
4,-73.6047,-40.5750
...,...,...
1242,-72.9781,-36.5932
1243,-73.1032,-36.7543
1244,-73.1032,-36.7543
1245,-73.1032,-36.7543
